In [ ]:
pip install PyMuPDF

In [ ]:
# 의사록 pdf 다운받기
import requests
from bs4 import BeautifulSoup
import os

if not os.path.exists('pdf_files'):
    os.makedirs('pdf_files')

for page_num in range(1, 41): 
    URL = f'https://www.bok.or.kr/portal/bbs/B0000245/list.do?menuNo=200761&pageIndex={page_num}'
    resp = requests.get(URL)
    soup = BeautifulSoup(resp.text, 'html.parser')
    
    li_list = []
    li = soup.find('div', class_='bdLine type2').find('ul').find('li')
    li_li = li.find_next_siblings('li')
    li_list.append(li)
    li_list.extend(li_li)
    
    for j in range(0, len(li_list)):
        try:
            link = li_list[j].find('div', class_='fileGoupBox').find('ul').find_all('li')[1].find('a').attrs['href']
            title = li_list[j].find('div', class_='row').find('span').find('a').find('span').text.strip()
            pdf_url = 'https://www.bok.or.kr' + link

            response = requests.get(pdf_url)
            with open(f'pdf_files/{title}.pdf', 'wb') as f:
                f.write(response.content)

            print(f"{title} 다운로드 완료")

        except Exception as e:
            print("Error:", e)

In [2]:
# tsv 파일로 생성

import fitz
import os
import pandas as pd
import re

def extract_date_from_filename(file_name):
    # 파일 이름에서 날짜 정보 추출
    match = re.search(r'(\d{4}\.\d{1,2}\.\d{1,2})', file_name)
    if match:
        return match.group(1)
    else:
        return None

folder_path = "./111"
tsv_file = "minutes22.tsv"

data = []
file_list = os.listdir(folder_path)

for file_name in file_list:
    file_path = os.path.join(folder_path, file_name)
    
    # 파일인 경우에만 처리
    if os.path.isfile(file_path) and file_name.lower().endswith('.pdf'):
        try:
            doc = fitz.open(file_path)
            # 파일 이름에서 날짜 정보 추출
            date_str = extract_date_from_filename(file_name)
            if date_str:
                try:
                    date = pd.to_datetime(date_str, format='%Y.%m.%d')
                except ValueError:
                    print("Invalid date format in file name:", file_name)
                    date = None
            else:
                print("Date information not found in file name:", file_name)
                date = None
            
            # 파일별로 텍스트 추출하여 데이터에 추가
            text = ""
            for page in doc:
                text += page.get_text()
                text.replace('\n', ' ')
            
            # 날짜와 텍스트를 데이터에 추가
            data.append({"날짜": date, "내용": text})
            doc.close()
        except Exception as e:
            print("Error processing file:", file_name)
            print(e)
            continue  # 오류 발생한 파일 건너뛰기

df = pd.DataFrame(data)
df.to_csv(tsv_file, sep='\t', index=False)

print("TSV 파일이 생성되었습니다:", tsv_file)

TSV 파일이 생성되었습니다: minutes22.tsv


In [ ]:
# json 파일로 생성

import fitz
import os
import pandas as pd
import re

def extract_date_from_filename(file_name):
    # 파일 이름에서 날짜 정보 추출
    match = re.search(r'(\d{4}\.\d{1,2}\.\d{1,2})', file_name)
    if match:
        return match.group(1)
    else:
        return None

folder_path = "./minutes_pdfs"
json_file = "minutes.json"

data = []
file_list = os.listdir(folder_path)

for file_name in file_list:
    file_path = os.path.join(folder_path, file_name)
    
    # 파일인 경우에만 처리
    if os.path.isfile(file_path) and file_name.lower().endswith('.pdf'):
        try:
            doc = fitz.open(file_path)
            # 파일 이름에서 날짜 정보 추출
            date_str = extract_date_from_filename(file_name)
            if date_str:
                try:
                    date = pd.to_datetime(date_str, format='%Y.%m.%d')
                except ValueError:
                    print("Invalid date format in file name:", file_name)
                    date = None
            else:
                print("Date information not found in file name:", file_name)
                date = None
            
            # 파일별로 텍스트 추출하여 데이터에 추가
            text = ""
            for page in doc:
                text += page.get_text()

            # 날짜와 텍스트를 데이터에 추가
            data.append({"날짜": date, "내용": text.replace('\n', ' ')})
            doc.close()
        except Exception as e:
            print("Error processing file:", file_name)
            print(e)
            continue  # 오류 발생한 파일 건너뛰기

df = pd.DataFrame(data)
df.to_json(json_file, orient='records', force_ascii=False)
print("JSON 파일이 생성되었습니다:", json_file)


In [35]:
df

,날짜,내용
0,2006-05-11,금융통화위원회 의사록 2006년도 제10차 회의 1. 일 자 2006년...
1,2006-05-25,금융통화위원회 의사록 2006년도 제11차 회의 1. 일 자 2006년...
2,2006-06-08,금융통화위원회 의사록 2006년도 제12차 회의 1. 일 자 2006년...
3,2006-06-22,금융통화위원회 의사록 2006년도 제13차 회의 1. 일 자 2006년...
4,2006-07-07,금융통화위원회 의사록 2006년도 제14차 회의 1. 일 자 2006년...
...,...,...
308,2023-04-11,2023년도 제7차 금융통화위원회(정기) 의사록 한 국 은 행 - 1 - 1. 일 ...
309,2024-01-11,2024년도 제1차 금융통화위원회(정기) 의사록 한 국 은 행 - 1 - 1. 일 ...
310,2024-01-25,2024년도 제2차 금융통화위원회(정기) 의사록 한 국 은 행 - 1 - 1. 일 ...
311,2024-02-08,2024년도 제3차 금융통화위원회(정기) 의사록 한 국 은 행 - 1 - 1. 일 ...
